In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import tensorflow as tf

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor, AutoTokenizer
from sklearn.utils import shuffle

In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/text-gen/dataset1.csv", index_col=[0],encoding='windows-1252')
train_df

,Input-text,Target-text
Prefix,,
webNLG,Bhuvneshwar|Wade|FOUR,"Bhuvneshwar to Wade, FOUR, brilliant shot!"
webNLG,Harshal Patel|Tim David|SIX,"Harshal Patel to Tim David, SIX, tonked! Right..."
webNLG,Bhuvneshwar|Wade|WIDE,"Bhuvneshwar to Wade, wide, attempts a wide yor..."
webNLG,Bhuvneshwar|Tim David|WIDE,"Bhuvneshwar to Tim David, wide, fires this dow..."
webNLG,Axar|Josh Inglis|Bowled,"Axar to Josh Inglis, out Bowled!!"
...,...,...
webNLG,Maxwell | Gambhir| FOUR,"Maxwell to Gambhir, FOUR, 18 runs from the over."
webNLG,Aaron | Narine| SIX,"Aaron to Narine, SIX, lovely shot."
webNLG,Aaron | Narine| SIX,"Aaron to Narine, SIX, back-to-back sixes."


In [ ]:
train_df=train_df.iloc[  :,:]
train_df = shuffle(train_df)
batch_size=2
num_of_batches=int(len(train_df)/batch_size)

In [ ]:
train_df

,Input-text,Target-text
Prefix,,
webNLG,Boult|Raina|FOUR,"Boult to Raina, FOUR, a well-controlled pull s..."
webNLG,Watson|Yuvraj|FOUR,"Watson to Yuvraj, FOUR, back to the usual. Is ..."
webNLG,"Madhevere|Edwards| no run,","Madhevere to Edwards, no run, tossed up full o..."
webNLG,D Kulkarni|Chris Lynn|SIX,"D Kulkarni to Chris Lynn, SIX, flat, strong, six!"
webNLG,McClenaghan|Smith|FOUR,"McClenaghan to Smith, FOUR, over the ropes"
...,...,...
webNLG,Tahir | Maxwell| SIX,"Tahir to Maxwell, SIX, the most powerful deliv..."
webNLG,Nehra | Roy| FOUR,"Nehra to Roy, FOUR, lovely shot, banged in sho..."
webNLG,Gony|Chris Lynn|FOUR,"Gony to Chris Lynn, FOUR, Chris Lynn and his g..."


In [ ]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [ ]:
tokenizer2 = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base",
                                             return_dict=True)
model.to(dev)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
from IPython.display import HTML, display

def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
num_of_epochs = 1

model.train()

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
    running_loss=0
    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():

            inp = 'WebNLG: '+row['Input-text']+'</s>'
            labels = row['Target-text']+'</s>'
            inputbatch.append(inp)
            labelbatch.append(labels)

        inputbatch=tokenizer2.batch_encode_plus(inputbatch,padding=True,max_length=1000,return_tensors='pt') ["input_ids"]
        labelbatch=tokenizer2.batch_encode_plus(labelbatch,padding=True,max_length=1000,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)
        optimizer.zero_grad()

        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:
          loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        loss.backward()

        optimizer.step()

    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))



Running epoch: 1


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1 , Running loss: 2.538730381750593


In [ ]:
import time
def generate(text):
  model.eval()
  input_ids = tokenizer2.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  # Batch size 1
  input_ids = input_ids.to(dev)
  s = time.time()
  outputs = model.generate(input_ids)
  gen_text=tokenizer2.decode(outputs[0]).replace('<pad>','').replace('</s>','')
  elapsed = time.time() - s
  print('Generated in {} seconds'.format(str(elapsed)[:4]))


  return gen_text

In [ ]:
generate("Hardik|kohli|SIX")

Saving and loading entire model

In [ ]:
torch.save(model, '/content/drive/MyDrive/Story_Generation/Sentence_gen/mod.pth')

In [ ]:
Model1 = torch.load('/content/drive/MyDrive/text-gen/Saved_model/mod3.pth')

In [ ]:
import time
def generate(text):
  model.eval()
  input_ids = tokenizer2.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  # Batch size 1
  input_ids = input_ids.to(dev)
  s = time.time()
  outputs = Model1.generate(input_ids)
  gen_text=tokenizer2.decode(outputs[0]).replace('<pad>','').replace('</s>','')
  elapsed = time.time() - s
  print('Generated in {} seconds'.format(str(elapsed)[:4]))


  return gen_text

In [ ]:
generate("Hardik | Kohli | SIX")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 7.08 seconds


'Hardik Pandya to Kohli, SIX, a salute in celebration'

In [ ]:
import transformers
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
def load_dataset(train_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset ,data_collator

train_dataset ,data_collator = load_dataset('/content/drive/MyDrive/GPT-2_output/sample4.txt', tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/GPT-2_output/GPT-2_gen",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16
    per_device_eval_batch_size=16,
    eval_steps = 400,
    save_steps=800,
    warmup_steps=500,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

generator = pipeline('text-generation',model='/content/drive/MyDrive/GPT-2_output/GPT-2_gen', tokenizer='gpt2')

In [ ]:
ab = generator(generate("Hardik | Kohli | out"))

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated in 0.97 seconds


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
ab=generator("Chahal to Axar, out Caught by Pollard!!")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
def listToString(ab):
  p=[]
  for key, val in ab[0].items():
    p.append("{}".format(val))
  str1 = ""
  for ele in p:
    str1 += ele
  res="";
  for ch in str1:
    if ch=='"' :
      break;
    else:
      res += ch;

  res+='.'
  return res

print(listToString(ab))  #printing the final commentary

In [ ]:
!pip install gTTs

In [ ]:
from gtts import gTTS
text_to_say = listToString(ab)
language = "en"
gTTS_object = gTTS(text = text_to_say,lang = language,slow = False)
gTTS_object.save("/content/drive/MyDrive/Text_to_speech_outputs/gtts.wav")

In [ ]:
from IPython.display import Audio
Audio("/content/drive/MyDrive/Text_to_speech_outputs/gtts.wav")#final audio commentary is in gtts.wav file